In [1]:
import os # miscellaneous os interfaces

In [2]:
import datetime as dt   # python core datetime utility module
import datetime_ as dt_ # https://ds-gringotts.readthedocs.io/en/latest/modules/utils/dtutils.html

In [3]:
from tqdm import tqdm as TQ # progress bar for loops

In [4]:
import pandas as pd

%precision 3
pd.set_option('display.max_rows', 50) # max. rows to show
pd.set_option('display.max_columns', 15) # max. cols to show
pd.options.display.float_format = '{:,.3f}'.format # float precisions

In [5]:
import forexrates # a simple interface to fetch data from multiple sources

In [6]:
start, end = dt.date(2015, 1, 1), dt.date(2015, 1, 5)
dates = list(dt_.date_range(start, end)) # please refer gist documentation

In [7]:
EXCHANGE_RATES_API_KEY = input("Enter API Key for Exchange Rates IO (exchangeratesapi.io): ") or os.getenv("EXCHANGE_RATES_API_KEY")

Enter API Key for Exchange Rates IO (exchangeratesapi.io): 89f9a1b9e6f36791a0544545c528d308


In [8]:
data = [
    forexrates.api.ExchangeRatesAPI(apikey = EXCHANGE_RATES_API_KEY, endpoint = date.strftime("%Y-%m-%d")).get()
    for date in TQ(dates, desc = f"Fetching FOREX Rates from {start} to {end} (= {(end - start).days + 1}):")
]

Fetching FOREX Rates from 2015-01-01 to 2015-01-05 (= 5):: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.99it/s]


In [9]:
parser = forexrates.io.dataframe.ExchangeRatesIO(data)
dataframe = parser.dataframe()

dataframe.sample(3)

Parsing Records...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 178.72it/s]


,effective_date,base_currency_code,target_currency_code,foreign_exchange_rate
306,2015-01-02,EUR,TTD,7.713
380,2015-01-03,EUR,GHS,3.854
93,2015-01-01,EUR,MMK,"1,244.774"
